In [1]:
# importing libraries 
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from torch.utils.data import ConcatDataset
import os
from datasets import Dataset
# import packages (remember to import the local_transoformers package in your folder correctly!)
from local_transformers_clinical_BERT.src.transformers import TrainingArguments
import pandas as pd
import torch
from datasets import Dataset
import tensorflow as tf
from local_transformers_clinical_BERT.src.transformers.models.CustomClinicalBERT import CustomClinicalBertForCombinedLearning # reads the local_transformers folder instead of the actual transformers package
from local_transformers_clinical_BERT.src.transformers.CustomComputeLoss import CustomTrainer # reads the local_transformers folder instead of the actual transformers package
import os
import pickle
import numpy as np
from xgboost import XGBClassifier
import numpy as np
from numpy import mean, std
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from tqdm import tqdm
from local_transformers_clinical_BERT.src.transformers.data.data_collator import DataCollatorForLanguageModeling
from local_transformers_clinical_BERT.src.transformers.models.auto.tokenization_auto import AutoTokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

# Reading data
df=pd.read_csv("df.csv")
df["death_in_30"] = df["death_in_30"].astype(int)
df["DVT"] = df["DVT"].astype(int)
df["PE"] = df["PE"].astype(int)
df["post_aki_status"] = df["post_aki_status"].astype(int)
df["PNA"] = df["PNA"].astype(int)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = CustomClinicalBertForCombinedLearning.from_pretrained("medicalai/ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=10) #NOTE: Change lambda constant to the appropriate one

# preparing the clinical notes for training and fine-tuning 
# this function here helps set up and prepare the data for fine-tuning, including those of the additional labels. 
def tokenize_and_prepare(batch, text_col, outcome_col, task_id):
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    encodings = tokenizer(batch[text_col], padding='max_length', truncation=True, max_length=164)
    additional_labels = torch.tensor(batch[outcome_col])  # make sure additional_labels are tensors
    task_ids = torch.tensor([task_id] * len(additional_labels))
    return {
        "input_ids": encodings["input_ids"], 
        "attention_mask": encodings["attention_mask"], 
        "additional_labels": additional_labels, 
        "labels": encodings["input_ids"], 
        "task_ids": task_ids
    }


# helping prepare the data per task given the multi-label nature of foundational models. 
def prepare_data_per_task(df, text_col, outcome_col, task_id):
    df = df.dropna(subset=[outcome_col])
    df[outcome_col]=df[outcome_col].astype(int)
    texts = list(df[text_col])
    outcomes = list(df[outcome_col])
    data = {text_col: texts, outcome_col: outcomes}
    dataset = Dataset.from_dict(data)
    dataset = dataset.map(lambda examples: tokenize_and_prepare(examples, text_col, outcome_col, task_id), batched=True, batch_size=164)
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "additional_labels", "labels", "task_ids"])
    return dataset


# stacking the data of different labels on top of one another so we can perform MTL fine-tuning
def stack_data(df, tasks):
    datasets = []
    for i, task in enumerate(tasks):
        dataset = prepare_data_per_task(df, "AN_PROC_NAME", task, i)
        datasets.append(dataset)
    combined_dataset = ConcatDataset(datasets)
    return(combined_dataset)

2023-11-04 07:53:29.542651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 07:53:30.017527: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-04 07:53:30.017571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-04 07:53:30.017576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.4.attention.v_lin.bias', 'distilbert.transformer.layer.3.attention.out_lin.bias', 'vocab_projector.weight', 'distilbert.transformer.layer.3.output_layer_norm.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.2.attention.out_lin.bias', 'distilbert.transformer.layer.5.attention.out_lin.weight', 'vocab_transform.weight', 'distilbert.transformer.layer.1.ffn.lin2.bias', 'distilbert.transformer.layer.5.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.2.attention.k_lin.weight', 'vocab_layer_norm.bias', 'distilbert.transform

In [2]:
print(len(df))

84875


In [3]:
# this finetunes our model
def get_model(name, train_dataset,val_dataset):
    print("here")
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    model = CustomClinicalBertForCombinedLearning.from_pretrained("medicalai/ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=10,num_tasks=6) #NOTE: Change lambda constant to the appropriate one
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=6,
        per_device_train_batch_size=48,
        per_device_eval_batch_size=48,
        warmup_steps=1500,
        weight_decay=0.001,
        learning_rate=0.00001,
        logging_dir="./logs",
        logging_steps=1000,
        save_strategy="epoch")
    # Define the data collator for masked language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
    # Instantiate the Trainer
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer)
    trainer.train()
    trainer.model.save_pretrained(name)
    trainer.tokenizer.save_pretrained(name)

In [4]:
# gets the text representation from the updated specified model
def get_clinicalBERT_updated(texts, name):
    notes = list(texts)
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = CustomClinicalBertForCombinedLearning.from_pretrained(name, output_hidden_states=True)

    # Check if CUDA is available and if so, move the model to GPU
    if torch.cuda.is_available():
        model = model.to("cuda")

    # Tokenize texts and create input IDs and attention masks
    input_ids, attention_masks = [], []
    for text in notes:
        encoded_input = tokenizer.encode_plus(text, max_length=128, padding='max_length', truncation=True, 
                                              return_attention_mask=True, return_tensors='pt')  # change max_length if needed [NOTE: max_length may differ across task]
        input_ids.append(encoded_input['input_ids'])
        attention_masks.append(encoded_input['attention_mask'])

    # Convert lists to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # If CUDA is available, move the tensors to GPU
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")
        attention_masks = attention_masks.to("cuda")

    # Batch size for processing
    batch_size = 32
    # Initialize an empty list to store embeddings
    embeddings = []
    # Process input IDs and attention masks in batches
    for i in tqdm(range(0, len(input_ids), batch_size)):
        # print("progress:", (i/len(input_ids))*100)
        input_ids_batch = input_ids[i:i+batch_size]
        attention_masks_batch = attention_masks[i:i+batch_size]

        with torch.no_grad():
            # Get the model's output
            output = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)

        # Retrieve the last_hidden_state from the output
        last_hidden_state = output.hidden_states[-1]

        # Optionally, you can obtain the pooled_output by averaging or max-pooling the last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)

        # If CUDA is available, move the embeddings back to CPU
        if torch.cuda.is_available():
            pooled_output = pooled_output.to("cpu")

        # Add the embeddings to the list
        embeddings.extend(pooled_output.numpy())

    embeddings = np.array(embeddings)
    return embeddings


In [5]:
# performs the 5-fold for XGBoost (default model)

def rounding(take_mean, take_std):
    return(round(take_mean, 2), round(take_std,2))

def K_fold_val(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores, auprc_scores,accuracy_scores, recall_scores,precision_scores, specificity_scores,f_scores = [],[],[],[],[],[],[]
    # define parameters for tuning
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            # train val split
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_clinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_clinicalBERT_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        # removing rows with potential NAs in them 
        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        

        # performing the XGboost classification
        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]
        y_pred_class = best_model.predict(X_test)

        # Compute the metrics
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)
        accuracy=accuracy_score(y_test, y_pred_class)
        recall = recall_score(y_test, y_pred_class)
        precision = precision_score(y_test, y_pred_class)
        # For specificity (True Negative Rate)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
        specificity = tn / (tn + fp)
        f_score=f1_score(y_test, y_pred_class)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        accuracy_scores.append(accuracy)
        recall_scores.append(recall)
        precision_scores.append(precision)
        specificity_scores.append(specificity)
        f_scores.append(f_score)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc,std_auroc = rounding(np.mean(auroc_scores), np.std(auroc_scores))
    mean_auprc, std_auprc = rounding(np.mean(auprc_scores), np.std(auprc_scores))
    mean_accuracy, std_accuracy=rounding(np.mean(accuracy_scores), np.std(accuracy_scores))
    mean_precision, std_precision=rounding(np.mean(precision_scores), np.std(precision_scores))
    mean_recall, std_recall=rounding(np.mean(recall_scores), np.std(recall_scores))
    mean_specificity, std_specificity=rounding(np.mean(specificity_scores), np.std(specificity_scores))
    mean_f1,std_f1=rounding(np.mean(f_scores), np.std(f_scores))  
    
    
    print_statement=f'''
    metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, 
    \n Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}, 
    \n Mean ACC:{mean_accuracy}, SD ACC:{std_accuracy}, 
    \n Mean precision:{mean_precision}, std precision:{std_precision}, 
    \n mean recall:{mean_recall}, std_recall:{std_recall},
    \n mean specificity:{mean_specificity}, std specificity:{std_specificity}, 
    \n mean f1:{mean_f1}, std f1:{std_f1} 
    '''
    return(print_statement)


In [6]:
# performs the 5-fold for LogReg
def K_fold_val_log_reg(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []
    # define parameters for tuning
    param_grid = {
    'C': [0.01, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'newton-cholesky']
    }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_clinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_clinicalBERT_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        model = LogisticRegression(random_state=42,max_iter=10000)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        X_train = X_train.astype(np.float64)
        y_train = y_train.astype(np.float64)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}_logreg.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [7]:
# performs the 5-fold for RF
def K_fold_val_rf(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []
    # define parameters for tuning
    param_grid = {
        'max_depth': [4, None],  # maximum depth of the tree
        'min_samples_split': [2, 5],  # minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 3],  # minimum number of samples required to be at a leaf node
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_clinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_clinicalBERT_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        model = RandomForestClassifier(random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}_rf.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [8]:
results=K_fold_val("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

1it [04:17, 257.79s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

2it [08:39, 259.87s/it]

...

3it [12:58, 259.64s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

4it [17:20, 260.75s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

5it [21:38, 259.75s/it]


In [12]:
results=K_fold_val_log_reg("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   4.7s
[CV] END ...................C=0.01, penal

In [ ]:
results=K_fold_val_rf("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.1s
[CV] END max_depth=4, min_sampl

1it [1:21:03, 4863.77s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_sampl

2it [2:43:55, 4927.06s/it]

...

3it [4:03:38, 4861.23s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_sampl

4it [5:24:30, 4857.90s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.2s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.2s
[CV] END max_depth=4, min_sampl

5it [6:44:44, 4856.85s/it]


## Part II: DVT

In [10]:
results=K_fold_val("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [03:56, 236.80s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [07:53, 236.64s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [11:47, 235.52s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

4it [15:39, 234.25s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [19:34, 234.92s/it]


In [8]:
results=K_fold_val_log_reg("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   3.3s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.1s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

In [10]:
results=K_fold_val_rf("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.5s
[CV] END max_depth=4, min_sampl

1it [1:34:28, 5668.70s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_sampl

2it [3:17:47, 5980.69s/it]

...

3it [4:56:22, 5950.65s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_sampl

4it [6:21:47, 5624.48s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_sampl

5it [7:52:49, 5673.95s/it]


## Part III: PE

In [12]:
results=K_fold_val("PE", df) 

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [03:43, 223.87s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

2it [07:32, 226.72s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [11:18, 226.16s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [15:03, 225.94s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [18:48, 225.69s/it]


In [12]:
results=K_fold_val_log_reg("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.4s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.1s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.2s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.3s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.4s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

In [14]:
results=K_fold_val_rf("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_sampl

1it [1:28:00, 5280.91s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_sampl

2it [3:06:32, 5652.15s/it]

...

3it [4:34:40, 5485.79s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_sampl

4it [5:56:06, 5248.79s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_sampl

5it [7:18:27, 5261.55s/it]


## Part 4: PNA

In [14]:
results=K_fold_val("PNA", df) 

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [03:54, 234.90s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [07:48, 234.36s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [11:40, 233.17s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [15:34, 233.57s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [19:27, 233.41s/it]


In [16]:
results=K_fold_val_log_reg("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.7s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.1s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   3.1s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.9s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

In [18]:
results=K_fold_val_rf("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.7s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.7s
[CV] END max_depth=4, min_sampl

1it [1:21:56, 4916.33s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_sampl

2it [2:50:18, 5143.46s/it]

...

3it [4:12:33, 5047.93s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_sampl

4it [5:32:18, 4944.32s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  30.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  29.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  29.8s
[CV] END max_depth=4, min_sampl

5it [7:02:25, 5069.17s/it]


## Part 5: Post_AKI_status

In [16]:
results=K_fold_val("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

1it [04:47, 287.64s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

2it [09:37, 288.92s/it]

...

3it [14:32, 291.50s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

4it [19:21, 290.52s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

5it [24:12, 290.46s/it]


In [20]:
results=K_fold_val_log_reg("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   5.1s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   8.4s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=  10.1s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   8.4s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

In [22]:
results=K_fold_val_rf("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  28.3s
[CV] END max_depth=4, min_sampl

1it [1:03:55, 3835.66s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.3s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_sampl

2it [2:04:19, 3711.32s/it]

...

3it [3:05:13, 3685.16s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_sampl

4it [4:06:35, 3683.69s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  28.4s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  28.4s
[CV] END max_depth=4, min_sampl

5it [5:07:24, 3688.95s/it]


## Part 6: delirium

In [18]:
results=K_fold_val("postop_del", df) 

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

1it [02:27, 147.27s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

2it [04:53, 146.37s/it]

...

3it [07:19, 146.54s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

4it [09:43, 145.42s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

5it [12:10, 146.03s/it]


In [24]:
results=K_fold_val_log_reg("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.2s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

In [26]:
results=K_fold_val_rf("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_sampl

1it [04:44, 284.72s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_sampl

2it [09:28, 284.40s/it]

...

3it [14:14, 285.00s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_sampl

4it [19:04, 286.78s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   3.6s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   3.6s
[CV] END max_depth=4, min_sampl

5it [23:51, 286.34s/it]
